### Preprocessing Notebook for VoxCeleb1 Data Downloaded from https://mm.kaist.ac.kr/datasets/voxceleb/

In [1]:
#imports
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import shutil
import zipfile
import tarfile

In [82]:
#paths
DATA_PATH = '/usr/xtmp/hs285'
AUDIO_PATH = os.path.join(DATA_PATH, 'audio/')
VIDEO_PATH = os.path.join(DATA_PATH, 'video/')
VIDEO_TXT_PATH = os.path.join(DATA_PATH, 'txt/')


In [ ]:
#rename directories (only need to do once)
os.rename(os.path.join(DATA_PATH, 'wav'), AUDIO_PATH)
os.rename(os.path.join(DATA_PATH, 'unzippedFaces'), VIDEO_PATH)

In [5]:
#change directory structure to remove redundance 1.6/ directory in video path
list_of_dirs = os.listdir(VIDEO_PATH)

for folder in tqdm(list_of_dirs):
    root = VIDEO_PATH + folder
    
    for folder2 in os.listdir(root + '/1.6/'):
        shutil.move(root + '/1.6/' + folder2, os.path.join(root, folder2))
        
    os.rmdir(root + '/1.6/')

100%|██████████| 1251/1251 [00:48<00:00, 25.71it/s]


In [216]:
#load metadata for VoxCeleb1 dataset
meta = pd.read_csv(os.path.join(DATA_PATH, 'vox1_meta.csv'), sep='\t')
meta.head()


,VoxCeleb1 ID,VGGFace1 ID,Gender,Nationality,Set
0,id10001,A.J._Buckley,m,Ireland,dev
1,id10002,A.R._Rahman,m,India,dev
2,id10003,Aamir_Khan,m,India,dev
3,id10004,Aaron_Tveit,m,USA,dev
4,id10005,Aaron_Yoo,m,USA,dev


In [84]:
#create dictionary mapping celebrity id to name
name2id = meta.set_index('VGGFace1 ID')['VoxCeleb1 ID'].to_dict()

In [10]:
#rename directories using ids instead of celebrity names
list_of_dirs = os.listdir(VIDEO_PATH)

for folder in tqdm(list_of_dirs):
    root = VIDEO_PATH + folder
    prev_name = folder
    new_name = name2id[folder]
    os.rename(VIDEO_PATH + prev_name, VIDEO_PATH + new_name)

100%|██████████| 1251/1251 [00:00<00:00, 2564.82it/s]


In [12]:
#restructure file directory so that audio and video file structures match
for src_dir, dirs, files in tqdm(os.walk(VIDEO_TXT_PATH)):
    dst_dir = src_dir.replace(VIDEO_TXT_PATH, VIDEO_PATH, 1)
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
    for file_ in files:
        src_file = os.path.join(src_dir, file_)
        dst_file = os.path.join(dst_dir, file_)
        if os.path.exists(dst_file):
            os.remove(dst_file)
        shutil.move(src_file, dst_dir)

23032it [03:52, 99.12it/s] 


In [4]:
pd.read_csv(os.path.join(DATA_PATH, 'iden_split.txt'))

,3 id10003/na8-QEFmj44/00003.wav
0,1 id10003/tCq2LcKO6xY/00002.wav
1,1 id10003/K5zRxtXc27s/00001.wav
2,1 id10003/bDxy7bnj_bc/00004.wav
3,1 id10003/E_6MjfYr0sQ/00011.wav
4,1 id10003/5ablueV_1tw/00010.wav
...,...
153510,1 id11251/s4R4hvqrhFw/00002.wav
153511,1 id11251/gFfcgOVmiO0/00006.wav
153512,3 id11251/7GtZpUtReJ8/00001.wav
153513,2 id11251/5-6lI5JQtb8/00001.wav


In [205]:
def find_celebrity_from_id(s):
    return list(name2id.keys())[list(name2id.values()).index(s)]

In [208]:
find_celebrity_from_id(testset.dataset[0].split('/')[0])

'Aamir_Khan'

In [214]:
find_celebrity_from_id('id10003')

'Aamir_Khan'

In [211]:
print('5 celebrities ')
for i in range(5):
    most_similar_audio = testset.dataset[score_index_a[i]+1].split('/')[0]
    print(find_celebrity_from_id(most_similar_audio))
print('5 celebrities most likely to look their voice sounds like Aamir Khan')
for i in range(5):
    most_similar_video = testset.dataset[score_index_v[i]+1].split('/')[0]
    print(find_celebrity_from_id(most_similar_video))
    


Robbie_Kay
Constance_Zimmer
Chris_Lowell
Scott_Porter
Seth_Rogen
5 celebrities most likely to look their voice sounds like Aamir Khan
Angela_Kinsey
Angela_Kinsey
Teri_Hatcher
Angela_Kinsey
Toby_Stephens


NameError: name 'testset' is not defined

In [ ]:
plt.imshow(v[0][0].permute(1, 2, 0))